## Cats vs Dogs
For this workshop you will be building a Convolutional neural network to classify cats vs dogs. You will need to be familiar with the theory of CNNs. Visit our lesson [here](http://caisplusplus.usc.edu/blog/curriculum/lesson7) for more info.

In [36]:
# Imports, make sure you have cv2 installed!
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

# You are not allowed to change any of these constants.

DATA_PATH = './data/'
TEST_PERCENT = 0.1
SELECT_SUBSET_PERCENT = 0.15

# The cat and dog images are of variable size.
RESIZE_WIDTH=32
RESIZE_HEIGHT=32
EPOCHS = 5

In [37]:
# Lets get started by loading the data.
# Make sure you have the data downloaded to ./data
# To download the data go to https://www.kaggle.com/c/dogs-vs-cats/data and download train.zip

X = []
Y = []

files = os.listdir(DATA_PATH)
# Shuffle so we are selecting about an equal number of dog and cat images.
shuffled_files = sklearn.utils.shuffle(files)
select_count = int(len(shuffled_files) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count)

subset_files_select = shuffled_files[:select_count]

DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    if 'cat' == input_file.split('.')[0]:
        Y.append(0.0)
    else:
        Y.append(1.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255



Going to load 3750 files
Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Train set has dimensionality (3375, 32, 32, 3)
Test set has dimensionality (375, 32, 32, 3)


In [ ]:
######################################
#TODO: (Optional)
# Perform any data preprocessing steps



######################################

### Defining the network
Here are some useful resources to help with defining a powerful network.
- Convolution layers (use the 2D convolution) https://keras.io/layers/convolutional/
- Batch norm layer https://keras.io/layers/normalization/
- Layer initializers https://keras.io/initializers/
- Dense layer https://keras.io/layers/core/#dense
- Activation functions https://keras.io/layers/core/#activation
- Regulizers: 
    - https://keras.io/layers/core/#dropout
    - https://keras.io/regularizers/
    - https://keras.io/callbacks/#earlystopping
    - https://keras.io/constraints/

In [61]:
######################################
#TODO:
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm
######################################

model = Sequential()

######################################
#TODO:
# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(RESIZE_WIDTH, RESIZE_HEIGHT, 3)))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))
######################################


######################################
#TODO:
# Define your loss and your objective
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
######################################

## Test Time
Now it's time to actually test the network. Don't change any of the parameters here except for the batch size.

Get above **65%**!

In [66]:
######################################
#TODO:
# Define the batch size
batch_size = 64
######################################
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 2700 samples, validate on 675 samples
Epoch 1/5
2700/2700 [==============================] - 12s - loss: 0.5337 - acc: 0.7270 - val_loss: 0.5944 - val_acc: 0.6741

In [67]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

375/375 [==============================] - 0s     

Got 66.40% accuracy
